# Yolo V4 on SageMaker Notebook

### Download pre-trained model and darknet code


In [ ]:
!rm -rf darknet
!git clone https://github.com/AlexeyAB/darknet
!cd darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

## Switch SageMaker notebook to p3.2xlarge

In [ ]:
!sudo bash -c 'echo "/usr/local/lib64" > /etc/ld.so.conf.d/opencv.conf'
!sudo ldconfig -v

In [ ]:
%%bash 
cd darknet 
make

In [ ]:
from IPython.display import Image
Image(filename='darknet/data/dog.jpg') 

In [ ]:
%%bash 
cp yolov4.weights darknet
cd darknet 

./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg -dont_show

In [ ]:
Image(filename='darknet/predictions.jpg') 